# MAIN CODE

In [1]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []
status = []

for page_number in range(0,1):  #125

    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B011&order=score&page={0}&request_id=2022519113331&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]
        sta = d['status']

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "11PRO" , '아이폰7', 'SE', 'se', '부품']
        targets = ['배터리', '판매', '팔아요','아이폰11', '아이폰 11']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # 위치
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)
                    # 판매 여부
                    if sta == '0' :           # 0: 판매중    3 : 판매완료
                        status.append('n')  # n 판매중
                    else : status.append('y')  # y 판매완료(예약완료 포함)

                    
                    

                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()
                
                    description.append(data['item_info']['description'])
                    


                    # print(name, price, unix, loc, sta)

        else:
            pass

# print(thunder1,thunder2, time, location )


# Desc 에서 배터리 효율 뽑아내기

In [ ]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)



['81', '82', 'NULL', 'NULL', '78', '86', 'NULL', '86', 'NULL', 'NULL', '83', '90', '77', '76', '74', '78', '86', '84', '87', 'NULL', '97', 'NULL', 'NULL', '80', '83', '79', '82', '78', 'NULL', '84', 'NULL', '97', '84', 'NULL', 'NULL', '80', '84', '83', 'NULL', 'NULL', '85', '79', '88', '85', '82', '81', '80', 'NULL', 'NULL', 'NULL', '82', '86', '79', '82', '93', '94', 'NULL', 'NULL', '94', '87', '77', '84', '83', '86', '79', 'NULL', '88', 'NULL', '84', '87', 'NULL', '85', '92', '85', '86', '100', '85', '84', '85', 'NULL', '86', 'NULL', '100', '85', 'NULL', '76', 'NULL', '84', '82', '79', '100', '82', '83', 'NULL', '94', '99', '87', 'NULL', '85', '83', '87', '86', '091', '86', 'NULL', '80', '84', 'NULL', 'NULL', 'NULL', 'NULL', '84', '85', '88', 'NULL', '78', '94', '88', 'NULL', 'NULL', 'NULL', '81', '100', '92', 'NULL', '85', '86', 'NULL', '78', '85', '89', '91', '80', 'NULL', '81', '81', '86', '81']


"\nfor i in range(len(li)):    \n    text = li[i]\n    regex2 = '\\d{2,3}'\n    pattern2 = re.compile(regex2)\n    res2 = pattern2.findall(text)\n    li1.append(res2)\n"

용량 빼내기

In [ ]:
size = []


for i in range(len(description)):
    text = description[i]
    text2 = thunder1[i]
    regex = '64|128'
    pattern =  re.compile(regex)
    res = pattern.findall(text)   #상품설명에서 용량 찾기
    res_size = pattern.findall(text2)  #네임에서 용량 찾기
    
    if len(res) == 0:
        if len(res_size) == 0 :
            size.append('NULL')
        else:
            size.append(res_size[0])
    else:
        size.append(res[0])
print(size)

['64', '128', '64', '128', '128', '64', '128', '128', '64', '64', '64', '128', '64', '64', '64', '128', '64', '64', '128', 'NULL', '64', '64', '128', '64', '64', '64', '128', '64', '128', '64', 'NULL', '64', '64', '64', '128', '128', '64', '128', '128', '128', '64', '64', 'NULL', '64', '128', '64', '64', '64', '64', '128', '64', '128', '64', '128', 'NULL', '128', '64', '128', '64', 'NULL', '64', '64', 'NULL', '128', '128', '64', '128', '64', '64', '128', '64', '64', '128', '128', '64', '64', 'NULL', '64', '128', '128', '128', 'NULL', '64', '64', '128', '64', '64', '128', '128', '128', '64', '64', '64', '64', '128', '128', '128', 'NULL', 'NULL', '128', '128', '64', '128', '64', '64', '128', '128', '64', '128', '64', 'NULL', '128', '64', '128', '128', '64', '128', 'NULL', '64', '64', '128', 'NULL', '64', '64', '128', '64', '128', '128', '128', '128', '64', '128', '64', 'NULL', '64', '128', 'NULL', '64']


# 판다스로 데이터프레임 만들기

In [ ]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size,
                  'status' : status
                  })
df

,name,price,time,loction,battery,size,status
0,애플 아이폰11 (A2221) 64G 블랙 B++급 단품,300000,2022-05,부산광역시,81,64,n
1,아이폰11 128gb,420000,2022-05,경기도,82,128,n
2,아이폰11 레드 64기가 A급 팔아요^^,424000,2022-05,광주광역시,NULL,64,n
3,아이폰 11 128,370000,2022-05,대구광역시,NULL,128,n
4,아이폰11 블랙 128GB 판매,360000,2022-05,서울특별시,78,128,n
...,...,...,...,...,...,...,...
133,아이폰 11 노랑이 팔아요.,350000,2022-05,경기도,NULL,NULL,n
134,아이폰11 64기가 화이트,270000,2022-05,서울특별시,81,64,n
135,아이폰11 128기가 퍼플,360000,2022-05,서울특별시,81,128,n
136,아이폰11 블랙 256기가,480000,2022-05,서울특별시,86,NULL,n


# 데이터 엑셀파일로 추출

In [ ]:
from pandas import ExcelWriter
df.to_excel('C:/Users/admin/Desktop/big/thunder.xlsx', index=False)